In [2]:
import matlab.engine
import numpy as np
from math import*
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from numba import jit,njit
import time

In [3]:
# Khởi động MATLAB engine
eng = matlab.engine.start_matlab()

In [4]:

def calculate_trajectory(map,startPoint,goalPoint):
    startPoint = matlab.double(startPoint) # [meters, meters, radians]
    goalPoint  = matlab.double(goalPoint)
    map  = matlab.double(map.tolist())
    result = eng.Hybrid_Astar(map,startPoint,goalPoint)
    result = np.array(result)
    point = result[:,:3]
    point[:,2] = (point[:,2]*180/pi)-90
    direction = result[:,3]
    return point, direction

In [5]:

map = cv2.imread('globalMapParking.png', cv2.IMREAD_GRAYSCALE)
print(map.shape)
@jit(nopython=True)
def create_binary_map(map_array):
    binary_map = np.copy(map_array)
    # Avoid multi-dimensional indexing by iterating over each element
    for i in range(binary_map.shape[0]):
        for j in range(binary_map.shape[1]):
            if binary_map[i, j] > 1:
                binary_map[i, j] = 1
            else :
                binary_map[i, j] = 0
    return binary_map
    


# Hàm callback khi chuột được click
def draw_circle(event, x, y, flags, param):
    global map
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(map, (x, y), 1, 255, -1)



# Tạo một cửa sổ OpenCV và thiết lập hàm callback
cv2.namedWindow('Draw Circle',cv2.WINDOW_NORMAL)
cv2.resizeWindow('Draw Circle', 300, 751)
cv2.setMouseCallback('Draw Circle', draw_circle)

while True:
    out = create_binary_map(map)
    cv2.imshow('Draw Circle', map)
    # Nhấn phím 'ESC' để thoát
    if cv2.waitKey(1) == 27:
        cv2.destroyAllWindows()
        break

(750, 150)


In [6]:

costValue = out

# Lấy giá trị từ các ô nhập liệu điểm mục tiêu


startPose = [130, 40, pi/2] # [meters, meters, radians]
goalPose  = [130,430,pi/2]

point_arr, dir = calculate_trajectory(map=costValue,startPoint= startPose, goalPoint= goalPose)




In [ ]:
# Tạo ma trận chuyển đổi từ hệ quy chiếu global sang hệ quy chiếu local
@jit(nopython=True)
def transformation_matrix(center,arr_point_in):
    array_out = np.array([])
    rotation_matrix = np.array([[1, 0 ],
                                    [0, -1],])
    translation_vector = np.array([-center[0]+100, center[1]+100])
    for i in range(0, len(arr_point_in)):
        point = np.dot(rotation_matrix, arr_point_in[i]) + translation_vector
        array_out = np.append(array_out,point,axis=0)
    return array_out

In [ ]:
# eng.quit()